# Datenanalyse der Brettspielwelt-DB

## 1. Datenbestand

Es stehen ca. 7 GB Logdateien vom Spiele-Portal "Brettspielwelt" zur Verfügung (seit 2007-01, alle Logdateien bis einschließlich 2025-07).
Jede Logdatei hat die Daten einer Partie.

Nach dem Download wurde jede Logdatei geparst (in ein Objekt eingelesen), validiert und in eine SQLite-DB gepumpt.
Dieser Vorgang dauerte ca. 20 Stunden (wobei die Indezies der DB aus Versehen bereits vor dem Import angelegt wurden - dadurch hat es schätzungsweise 4 Stunden länger gedauert):

```
C:\Users\frank\Source\PyCharm\tichu\.venv\Scripts\python.exe C:\Users\frank\Source\PyCharm\tichu\bin\bsw_import.py --ym1 2007-01 --ym2 2025-07
BSW Importer
Ab Datum: 2007-01
Bis Datum: 2025-07
Zip-Archiven: C:\Users\frank\Source\PyCharm\tichu\data\bsw\tichulog
SQLite-Datenbank: C:\Users\frank\Source\PyCharm\tichu\data\bsw\bsw.sqlite
Importiere Log-Dateien: 100%|██████████| 2411514/2411514 [19:47:30<00:00, 33.85 Datei/s, Logdateien=2411514, Leere=13964, Fehler=79663, Datei=202507/2411561.tch]
Aggregiere Daten für Spieler: 100%|██████████| 128462/128462 [42:27<00:00, 50.43 Spieler/s]
fertig

Process finished with exit code 0
```

Dabei wurde gezählt:
*   2.411.514 Logdateien insgesamt
*   13.964 leere Logdateien (ohne verwertbare Daten)

Die SQLite-Datenbankdatei ist 17.085.088 KB = 16,294 GB groß.

### 1.3 Anzahl Partien

*   2.397.550 Partien insgesamt
*   2.317.887 Partien fehlerfrei (vorher: 2.139.529, Diff: 178.358, darin waren 155747 PLAYED_CHANGED,
bleibt Diff=22.611, darunter 21.267 PHOENIX_LOW, bleibt 1.344

In [1]:
%%sql
-- Anzahl Partien gesamt
select count(*) as Partien from games;

,Partien
0,2397550


In [2]:
%%sql
-- Anzahl Partien groupiert nach Fehlern
select g.error_code as Code, e.name as Fehler, count(*) as Anzahl, e.description as Beschreibung
from games as g
left join errors as e on g.error_code = e.code
group by g.error_code
order by g.error_code;

,Code,Fehler,Anzahl,Beschreibung
0,0,NO_ERROR,2317887,Kein Fehler
1,70,GAME_NOT_FINISHED,66718,Partie nicht zu Ende gespielt.
2,71,GAME_OVERPLAYED,1907,"Ein oder mehrere Runden gespielt, obwohl die P..."
3,80,ROUND_FAILED,11038,Mindestens eine Runde ist fehlerhaft.


### 1.2 Anzahl Runden

*   22.458.253 Runden insgesamt.
*   22.437.758 Runden fehlerfrei (vorher 22.414.499, Diff = 23.259, darunter 21.267 PHOENIX_LOW, bleibt 1992, darunter 2018 Tichu-Suizid, bleibt -26 Überlappung mit SCORE-Fehler)
*   22.000.517 Runden unter den fehlerfreien Partien (durchschnittlich 9,5 Runden / Partie). (vorher 20.169.053)
*   88.002.068 "Perspektiv-Runden"

In [3]:
%%sql
-- Anzahl Runden gesamt
select count(*) as Runden from rounds;

,Runden
0,22458253


In [4]:
%%sql
-- Anzahl Runden groupiert nach Fehlern
select r.error_code as Code, e.name as Fehler, count(*) as Anzahl, e.description as Beschreibung
from rounds as r
left join errors as e on r.error_code = e.code
group by r.error_code
order by r.error_code;

,Code,Fehler,Anzahl,Beschreibung
0,0,NO_ERROR,22437758,Kein Fehler
1,12,DUPLICATE_CARD,10,Karten mehrmals vorhanden.
2,21,WISH_NOT_FOLLOWED,209,Wunsch nicht beachtet.
3,23,PLAYER_NOT_ON_TURN,2,Der Spieler ist nicht am Zug.
4,25,HISTORY_TOO_LONG,60,"Karten ausgespielt, obwohl die Runde vorbei is..."
5,31,DRAGON_GIVEN_TO_OWN_TEAM,306,Drache an eigenes Team verschenkt.
6,50,ANNOUNCEMENT_NOT_POSSIBLE,286,Tichu-Ansage an der geloggten Position nicht m...
7,60,SCORE_NOT_POSSIBLE,17258,Rechenfehler! Geloggtes Rundenergebnis ist nic...
8,61,SCORE_MISMATCH,2364,Geloggtes Rundenergebnis stimmt nicht mit dem ...


In [5]:
%%sql
-- Anzahl Fälle, wo der Phönix den niedrigeren Rang mimt
select count(*) from rounds where is_phoenix_low = 1

,count(*)
0,21267


In [6]:
%%sql
-- Tichu-Ansage, obwohl ein Mitspieler bereits fertig ist
select count(*) from players_rounds where is_tichu_suicidal = 1

,count(*)
0,2034


In [7]:
%%sql
-- Anzahl Runden unter den fehlerfreien Partien
select count(*) as Runden
from rounds as r
inner join games as g on g.id = r.game_id
where g.error_code = 0;

,Runden
0,22000517


In [8]:
# Runden unter fehlerfreien Partien / fehlerfreie Partien
22000517 / 2317887

9.49162620956069

In [9]:
# "Perspektiv-Runden" = Runden unter fehlerfreien Partien * 4
22000517 * 4

88002068

### 1.3 Anzahl Spieler

*   128.462 Spieler insgesamt
*   110.869 Spieler, die eine fehlerfreie Partie durchgespielt haben

In [10]:
%%sql
-- Anzahl Spieler gesamt
select count(*) as Spieler from players;

,Spieler
0,128462


In [11]:
%%sql
-- Anzahl Spieler, die eine fehlerfreie Partie durchgespielt haben
select count(distinct pr.player_id)
from games as g
inner join rounds as r on g.id = r.game_id
inner join main.players_rounds as pr on r.id = pr.round_id
where g.error_code = 0


,count(distinct pr.player_id)
0,125648


### 1.4 Anzahl Spielzüge und Stiche

Erste Auszählung mittel replay():
*   1.139.465.380 Spielzüge (über alle fehlerfreien Partien)
*   1.139.465.380 / 20.186.515 = 56,447 Spielzüge pro Runde


Im Spiel mit 4 HeuristicAgents fallen ca. 10.8 Stiche/Runde (so häufig wurden Karten kassiert).

Ich schätze 6 Spielzüge (Kartenlegen oder Passen) pro Stich. Das ergibt 64.8 Spielzüge pro Runde.

Bezogen auf die Perspektiv-Runden habe ich geschätzt **970.000.000 Stiche** bzw. **5.821.000.000 Spielzüge**.


## 2. Rundenergebnis skalieren
